In [1]:
import numpy as np
import pandas as pd
import json
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
COLOR_PALETTE = ['#d0e3f5',
                 '#712e67',
                 '#267592',
                 '#5fb12a',
                 '#fac800',
                 '#ff7917',
                 '#e23a34']

In [52]:
curves = ["curve_1", "curve_2"]

def load_curve(filename):
    return np.load('pointClouds/'  + filename + '.npy')

# Communities partition

In [55]:
COMMS = {}
with open('communities.json') as fh:
    PARTITIONS = json.load(fh)
    for curve in curves:
        partition = PARTITIONS[curve]
        COMMS[curve] = {c:[] for c in set(partition)}
        for vertex, c in enumerate(partition):
            COMMS[curve][c].append(vertex)

In [61]:
def community_adjacency(curve, scope='plot'):
    cs = COMMS[curve]
    n = np.sum([len(cs[k]) for k in cs.keys()])
    matrix = np.zeros((n,n))

    color = 1
    for group in cs.keys():
        if len(cs[group]) >1:
            for i in cs[group]:
                for j in cs[group]:
                    if scope == 'plot':
                        matrix[i,j] = color
                    else:
                        matrix[i,j] = 1
            color = color+1

    return(matrix) 


def plot_chain_with_communities(filename):
    community = COMMS[filename]
    curve = load_curve(filename)
    df = pd.DataFrame.from_records(curve, columns=['X', 'Y', 'Z'])
    
    nodes = communities_color_code(community)
    df["color"] = nodes    
        
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
    x=df['X'], y=df['Y'], z=df['Z'], 
    name = 'Curve',    
    marker=dict(
        size=8,
        color = df["color"],
        colorscale=COLOR_PALETTE,
        line=dict(width=4, 
                  color='DarkSlateGrey')
    ),
        
    line=dict(
        width=8,
        color = df["color"],
        colorscale=COLOR_PALETTE
    )),
    )
    fig.update_layout(scene=dict(xaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False,),
    yaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
               zaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
        camera=dict(
            up=dict(
                x=-10,
                y=0,
                z=30
            ),
            eye=dict(
                x=0.9,
                y=0.9,
                z=1.3,
            )
        )))
                  
    fig.update_layout(scene = dict(
                    xaxis_title=' ',
                    yaxis_title=' ',
                    zaxis_title=' '))
    fig.update_traces(hoverinfo="text",hovertemplate=nodes)

    return fig


## Visualising spatial modules given by communities

To improve visualisation, points in communities formed by single points are considered as unclassified and are all coloured in light blue

In [62]:
def communities_color_code(comms):
    n = np.sum([len(comms[k]) for k in comms.keys()])
    nodes = n*[0]
    color = 1
    for k in comms.keys():
        if len(comms[k])>1:
            for j in comms[k]:
                nodes[j] = color
            color = color+1
    return nodes

In [63]:
fig = plot_chain_with_communities('curve_1')
fig.show()

In [64]:
fig = plot_chain_with_communities('curve_2')
fig.show()

In [65]:
M = community_adjacency('curve_2')
px.imshow(M, color_continuous_scale= COLOR_PALETTE)

# Node centrality

In [43]:
def load_node_centrality(filename):
    
    return pd.read_csv('nodeCents/' + filename + '.tsv', sep="\t", header=None).values


def plot_chain_with_centrality(filename):
    community = COMMS[filename]
    curve = load_curve(filename)
    df = pd.DataFrame.from_records(curve, columns=['X', 'Y', 'Z'])
    
    nodes = load_node_centrality(filename)
    df["color"] = nodes    
        
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
    x=df['X'], y=df['Y'], z=df['Z'], 
    name = 'Curve',    
    marker=dict(
        size=8,
        color = df["color"],
        line=dict(width=4, 
                  color='DarkSlateGrey')
    ),
        
    line=dict(
        width=8,
        color = df["color"],
    )),
    )
    fig.update_layout(scene=dict(xaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False,),
    yaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
               zaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor='rgba(0,0,0,0)',
                         showbackground=False,
                         zerolinecolor='rgba(0,0,0,0)',showticklabels=False),
        camera=dict(
            up=dict(
                x=-10,
                y=0,
                z=30
            ),
            eye=dict(
                x=0.9,
                y=0.9,
                z=1.3,
            )
        )))
                  
    fig.update_layout(scene = dict(
                    xaxis_title=' ',
                    yaxis_title=' ',
                    zaxis_title=' '))

    return fig

In [44]:
fig = plot_chain_with_centrality('curve_1')
fig.show()

In [45]:
fig = plot_chain_with_centrality('curve_2')
fig.show()